In [36]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from scipy.stats import kurtosis, skew

In [37]:
# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [73]:
df_train = pd.read_csv('./data/X_train/X_train',index_col=0)
df_test = pd.read_csv('./data/X_test/X_test',index_col=0)
# 区分字母和命令
y_train_is = pd.read_csv('./data/y_train.csv', header=None, names=['is_letter'])
letter_sample = df_train[y_train_is.is_letter.values]
# label标签
y_train = letter_sample.iloc[:, -1].values
y_test = df_test.iloc[:, -1].values

scaler = StandardScaler()
X_train = scaler.fit_transform(letter_sample.to_numpy()[:,:-1])
X_test = scaler.transform(df_test.to_numpy()[:,:-1])
pca = PCA(n_components=1026)  # 选择降至5维，可根据需要调整
X_train = pca.fit_transform(X_train)  # 排除目标变量
X_test = pca.transform(X_test)

print(X_train.shape, X_test.shape,y_train.shape, y_test.shape)

(1919, 1026) (3069, 1026) (1919,) (3069,)


In [74]:

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# 定义数据集和数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [75]:
class MLP(nn.Module):
    def __init__(self, input_size=1026, hidden_size=64, num_classes=9, dropout_prob=0.5):
        super(MLP, self).__init__()
        # 第一层将输入维度从 36 减少到 32
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        # 第二层从 32 增加到给定的隐藏层大小
        self.fc2 = nn.Linear(512, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        # 第三层保持隐藏层大小不变
        self.fc3 = nn.Linear(hidden_size, 64)
        self.bn3 = nn.BatchNorm1d(64)
        # 第四层从隐藏层大小减少到 16
        self.fc4 = nn.Linear(64, 16)
        self.bn4 = nn.BatchNorm1d(16)
        # 最后一层从 16 减少到输出类别数
        self.fc5 = nn.Linear(16, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc5(x)
        return x



In [76]:
input_size = X_train.shape[1]
hidden_size = 128  # 隐藏层神经元数量
num_classes = 9  # 类别数（多分类）
learning_rate = 0.01
num_epochs = 500

# 初始化模型并将其移动到GPU
model = MLP(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 多分类任务用 CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [77]:
# 训练模型
best_accuracy = 0.0
all_preds = []
all_labels = []
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.append(predicted)
        all_labels.append(labels)
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    # 保存最好的模型
    if epoch_acc > best_accuracy:
        best_accuracy = epoch_acc
        torch.save(model.state_dict(), f'model_best.pth')
        print("Saved Best Model")

Epoch [1/500], Loss: 2.1119, Accuracy: 17.40%
Saved Best Model
Epoch [2/500], Loss: 1.9371, Accuracy: 22.30%
Saved Best Model
Epoch [3/500], Loss: 1.8317, Accuracy: 23.61%
Saved Best Model
Epoch [4/500], Loss: 1.7419, Accuracy: 27.93%
Saved Best Model
Epoch [5/500], Loss: 1.6742, Accuracy: 27.77%
Epoch [6/500], Loss: 1.6283, Accuracy: 27.05%
Epoch [7/500], Loss: 1.5735, Accuracy: 29.29%
Saved Best Model
Epoch [8/500], Loss: 1.5314, Accuracy: 30.85%
Saved Best Model
Epoch [9/500], Loss: 1.5204, Accuracy: 31.21%
Saved Best Model
Epoch [10/500], Loss: 1.4535, Accuracy: 32.78%
Saved Best Model
Epoch [11/500], Loss: 1.4275, Accuracy: 33.61%
Saved Best Model
Epoch [12/500], Loss: 1.4194, Accuracy: 32.93%
Epoch [13/500], Loss: 1.3891, Accuracy: 34.71%
Saved Best Model
Epoch [14/500], Loss: 1.4053, Accuracy: 34.03%
Epoch [15/500], Loss: 1.3710, Accuracy: 35.17%
Saved Best Model
Epoch [16/500], Loss: 1.3435, Accuracy: 35.17%
Epoch [17/500], Loss: 1.3896, Accuracy: 36.48%
Saved Best Model
Epoch 

In [80]:
# 评估模型
model.eval()  # 设置为评估模式
correct = 0
total = 0

with torch.no_grad():  # 禁用梯度计算，节省计算和内存使用
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 确保数据也在正确的设备上
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        correct += (predicted == labels).sum().item()

# 将列表中的所有Tensor连接成一个Tensor


# 计算准确率
accuracy = correct / total *100
print(f'Accuracy: {accuracy:.4f}')

# 如果需要，可以将Tensor转换为NumPy数组进行进一步的处理


# 可以进一步分析，如计算混淆矩阵等


Accuracy: 10.0684
